## Hotels

In [4]:
import sys
import os
import dotenv
import pandas as pd

sys.path.append("../src")
from hotelfunc import main, get_location_ids

sys.path.append("..")

dotenv.load_dotenv()

True

In [5]:
tk1 = os.getenv('rapidapi_token')

In [9]:
destinations = ["budapest", "milan"]
loc_ids = get_location_ids(destinations, tk1)
print(loc_ids)

['-850553', '-121726']


In [12]:
final_df = await main(loc_ids, tk1)

100%|██████████| 2/2 [01:15<00:00, 37.84s/it]


In [15]:
combined_df = pd.concat(final_df)

combined_df[combined_df["city"] == 'Milan'].to_csv("../datos/accommodations_milan.csv")
combined_df[combined_df["city"] == 'Budapest'].to_csv("../datos/accommodations_budapest.csv")

In [14]:
combined_df.shape

(1760, 10)